# Wikidata Authentication Examples for GKC

This notebook demonstrates how to authenticate to Wikidata and other Wikimedia projects using the GKC `WikiverseAuth` class. You'll learn:

- How to authenticate using bot passwords from environment variables
- How to make authenticated API requests to Wikidata
- How to get CSRF tokens for write operations
- How to target different Wikimedia projects (Wikipedia, Commons, etc.)
- How to work with custom MediaWiki instances

**Before running these examples**, you'll need to set up your bot credentials. See the [Authentication Guide](/gkc/authentication/) for detailed setup instructions.

## Setup

First, import the authentication module and handle exceptions:

In [ ]:
from gkc import WikiverseAuth, AuthenticationError
import os

## Example 1: Basic Authentication with Environment Variables

The most secure way to authenticate is using environment variables. Set these in your shell before running this notebook:

```bash
export WIKIVERSE_USERNAME="YourUsername@YourBot"
export WIKIVERSE_PASSWORD="your_bot_password"
```

Then create a `WikiverseAuth` object and authenticate:

In [ ]:
try:
    # Create auth object from environment variables
    auth = WikiverseAuth()
    
    # Check if credentials are available
    if not auth.is_authenticated():
        print("⚠ No credentials found in environment variables.")
        print("Set WIKIVERSE_USERNAME and WIKIVERSE_PASSWORD to try this example.")
    else:
        print("Authenticating to Wikidata...")
        print(f"Username: {auth.username}")
        print(f"API URL: {auth.api_url}")
        
        # Login to establish session
        auth.login()
        print("✓ Successfully logged in!")
        
        # Retrieve user information
        response = auth.session.get(
            auth.api_url,
            params={
                "action": "query",
                "meta": "userinfo",
                "uiprop": "rights|groups",
                "format": "json",
            },
        )
        
        user_info = response.json()
        if "query" in user_info and "userinfo" in user_info["query"]:
            user = user_info["query"]["userinfo"]
            print("\nAuthenticated User Info:")
            print(f"  Account Name: {user.get('name')}")
            print(f"  Groups: {', '.join(user.get('groups', []))}")
            print(f"  Permissions: {len(user.get('rights', []))} total rights")
        
        # Logout when done
        auth.logout()
        print("\n✓ Logged out successfully")

except AuthenticationError as e:
    print(f"❌ Authentication error: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

## Example 2: Using Custom MediaWiki Instances

You can authenticate to any MediaWiki instance, not just Wikidata. Just pass your credentials and the API URL:

In [ ]:
# Example showing how to specify credentials and custom API URL
# Note: Using example values here - replace with your actual instance details

auth_custom = WikiverseAuth(
    username="YourUsername@YourBot",
    password="your_bot_password",
    api_url="https://custom.wiki.org/w/api.php"
)

print("Custom MediaWiki Instance Configuration:")
print(f"  Username: {auth_custom.username}")
print(f"  API URL: {auth_custom.api_url}")
print("\nTo authenticate, you would call:")
print("  auth_custom.login()")
print("\nNote: This example uses placeholder credentials.")

## Example 3: Targeting Different Wikimedia Projects

While credentials are typically shared across Wikimedia projects, you can direct requests to different projects:

In [ ]:
# Different Wikimedia projects you can target
projects = {
    "Wikidata": "wikidata",
    "Wikipedia (English)": "wikipedia",
    "Wikimedia Commons": "commons",
    "Wikibooks": "wikibooks",
    "Wikiversity": "wikiversity",
}

print("Wikimedia Project Endpoints:")
print("-" * 60)

for project_name, api_shortcut in projects.items():
    # Create auth object pointing to each project
    auth_project = WikiverseAuth(
        username="User@Bot",  # Placeholder
        password="password",   # Placeholder
        api_url=api_shortcut
    )
    print(f"{project_name:25s} -> {auth_project.api_url}")

print("\nNote: Use the same credentials for all Wikimedia projects.")

## Example 4: Getting CSRF Tokens for Write Operations

Most write operations (creating items, editing, etc.) require a CSRF token. The `get_csrf_token()` method handles this automatically:

In [ ]:
try:
    auth = WikiverseAuth()
    
    if not auth.is_authenticated():
        print("⚠ No credentials available to demonstrate CSRF token retrieval.")
        print("\nHow to use CSRF tokens for write operations:")
        print("""
# 1. Login to establish session
auth.login()

# 2. Get CSRF token
csrf_token = auth.get_csrf_token()

# 3. Use token in edit operations
edit_params = {
    "action": "edit",
    "title": "Test:Page",
    "text": "Page content",
    "token": csrf_token,
    "format": "json"
}
response = auth.session.post(auth.api_url, data=edit_params)

# 4. Logout when done
auth.logout()
        """)
    else:
        print("Demonstrating CSRF token retrieval...")
        auth.login()
        print("✓ Logged in")
        
        # Get CSRF token
        csrf_token = auth.get_csrf_token()
        print(f"✓ CSRF Token obtained: {csrf_token[:30]}...")
        
        print("\nToken information:")
        print(f"  Length: {len(csrf_token)} characters")
        print(f"  Used for: Creating items, editing, deleting, etc.")
        
        auth.logout()
        print("\n✓ Logged out")

except AuthenticationError as e:
    print(f"❌ Authentication error: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

## Key Takeaways

### Authentication Setup
- **Environment Variables**: Most secure method - set `WIKIVERSE_USERNAME` and `WIKIVERSE_PASSWORD`
- **Direct Specification**: Pass credentials to `WikiverseAuth()` constructor if needed
- **Custom Instances**: Support for any MediaWiki installation with full URL

### Authentication Flow
1. Create `WikiverseAuth` object
2. Call `login()` to establish session
3. Use `session` for authenticated API requests
4. Call `logout()` when finished

### Common Tasks
- **Get user info**: Query `action=query&meta=userinfo`
- **Write operations**: Get CSRF token with `get_csrf_token()`
- **Target projects**: Use shortcut (`wikidata`, `wikipedia`, `commons`) or full URL

### Security Best Practices
- Never commit credentials to version control
- Use bot passwords (not your main password)
- Always call `logout()` when done
- Use environment variables in production/shared environments
- Remember Wikimedia username format: `Username@Botname`

### Next Steps
- See [Authentication Guide](/gkc/authentication/) for setup details
- Explore [Item Creation](wikidata_item_creation.ipynb) for using auth with Wikidata edits